In [1]:
# import packages 
from collections import Counter, defaultdict
import csv
import itertools
import numpy as np
import pandas as pd
import re
import sys

from khaiii import KhaiiiApi
from tqdm import tqdm

In [2]:
# tagging by Khaiii
api = KhaiiiApi()

In [3]:
# read texts
df_contents = pd.read_excel(f"./target/freq190312_texts.xlsx", dtype={'texts':str}) #폴더경로 설정필요

In [4]:
df_contents.head()

,texts
0,중국의 민간 기업이 미사일 개발에 필요한 소재를 북한에 수출했다고 아사히신문이 현지...
1,국제사회의 제재와 우려에도 불구하고 북한이 6차 핵실험을 감행했습니다. 한반도의 긴...
2,"과학기술정보통신부는 오늘 오전 9시10분쯤 지구 자기장, 즉 지자기의 강력한 교란이..."
3,중국의 대학 연구팀이 지난 3일 북한의 6차 핵실험 위력을 지진 관측 자료를 통해서...
4,김정은 집권 이후 북한이 부쩍 강조하고 있는 분야가 있습니다. 바로 과학기술입니다....


In [5]:
def lines2line(text):
    sentences = []
    text = text.split('.')
    #print(text)
    for t in text:
        t = re.sub(r'[?/~,""()]', '', t)
        t = t.strip()
    #    print(t)
        if t == '':
            continue
           
        sentences.append(t)
        
    return sentences


def k_analyze(sentences):
    tags = []
    for sentence in sentences:
        # tag = []
        for word in api.analyze(sentence):
            #print(word.lex)
            # word.lex 와 단어가 일치하면 저장
            if word.lex in e_w:
                tags.append(word.lex)
            # 형태소가 1개이고 명사면 저장
            elif (len(word.morphs)) == 1 and word.morphs[0].tag in nouns:
            #    print(word.lex)
                tags.append(word.lex)
            # 사전의 morphs와 단어가 일치하면 수정 
            elif (len(word.morphs)) > 1:
                w1 = [str(x) for x in word.morphs]
            #    print(w1)
                used = False
                for w in e_w:
                    w2 = [str(x) for x in api.analyze(w)[0].morphs]
                    if set(w2) <= set(w1):
            #            print(w)
                        tags.append(w)
                        used = True 
                        break
                    else:
                        continue
                if used == False:
                    for w3 in word.morphs:
                        if w3.tag in nouns:
                            tags.append(w3.lex)
            # 나머진 패스
            else:
                pass
    return tags   


def morph_analyze(sentences):
    morphs = []
    morphs_seperate = []
    for sentence in sentences:
        for word in api.analyze(sentence):
            m = str(word)
            morphs.append(m)
            for w in word.morphs:
                w = str(w)
                morphs_seperate.append(w)
    
    return morphs, morphs_seperate

In [6]:
nouns = ['NNP', 'NNG', "NNG"]

In [7]:
df_contents['lines'] = df_contents['texts'].apply(lambda x: lines2line(x))

In [8]:
df_contents.head(5)

,texts,lines
0,중국의 민간 기업이 미사일 개발에 필요한 소재를 북한에 수출했다고 아사히신문이 현지...,[중국의 민간 기업이 미사일 개발에 필요한 소재를 북한에 수출했다고 아사히신문이 현...
1,국제사회의 제재와 우려에도 불구하고 북한이 6차 핵실험을 감행했습니다. 한반도의 긴...,"[국제사회의 제재와 우려에도 불구하고 북한이 6차 핵실험을 감행했습니다, 한반도의 ..."
2,"과학기술정보통신부는 오늘 오전 9시10분쯤 지구 자기장, 즉 지자기의 강력한 교란이...",[과학기술정보통신부는 오늘 오전 9시10분쯤 지구 자기장 즉 지자기의 강력한 교란이...
3,중국의 대학 연구팀이 지난 3일 북한의 6차 핵실험 위력을 지진 관측 자료를 통해서...,[중국의 대학 연구팀이 지난 3일 북한의 6차 핵실험 위력을 지진 관측 자료를 통해...
4,김정은 집권 이후 북한이 부쩍 강조하고 있는 분야가 있습니다. 바로 과학기술입니다....,"[김정은 집권 이후 북한이 부쩍 강조하고 있는 분야가 있습니다, 바로 과학기술입니다..."


In [9]:
# Combine all contents
all_contents = []
for text in df_contents['texts']:
    text = text.split('.')
    for t in text:
        t = re.sub(r'[?/~,""()]', '', t)
        t = t.strip()
        if t == '':
            continue
    
        all_contents.append(t)

In [10]:
# make dictionary
df_dict = pd.read_excel("./target/freq190312_dict.xlsx", hearder=None)
df_dict = df_dict.rename(columns={'단어': 'words', '빈도': 'frequency'})
dict_ls = [i for i in df_dict['words']]

In [11]:
complex_words = []
dict_words = defaultdict(list)
fixed_words = []
for word in dict_ls:
    if '(' in word:
        dict_words['complex_words'].append(word)
        complex_words.append(word)
        w = re.sub(r'[()]', '', word)
        fixed_words.append(w)
    else:
        dict_words[str(len(word))].append(word)
        fixed_words.append(word)

### NLP

In [12]:
# 기분석 사전 오류수정 
error_words = []

with open("errors.txt", 'r') as e: 
    lines = e.readlines()
    for line in lines:
        line = line.split("with")
        if len(line) == 1:
            continue
        else: 
            w = line[1]
            o, w = w.split(":")
            w = w.split("\t")[0]
            o = o.split("\t")[1]
            w = w.replace('"', '').strip()
            o = o.replace('"', '').strip()
            error_word = (w, o)
            error_words.append(error_word)

In [13]:
# 기분석 사전 만들기
dict_ls.sort(key=len)
e_w = [x[0] for x in error_words] 
dict_ls_fixed = [x for x in dict_ls if x not in e_w]

In [14]:
df_contents['tags'] = np.nan
df_contents['morphs'] = np.nan
df_contents['mor_sep'] = np.nan

In [15]:
df_contents.head(5)

,texts,lines,tags,morphs,mor_sep
0,중국의 민간 기업이 미사일 개발에 필요한 소재를 북한에 수출했다고 아사히신문이 현지...,[중국의 민간 기업이 미사일 개발에 필요한 소재를 북한에 수출했다고 아사히신문이 현...,NaN,NaN,NaN
1,국제사회의 제재와 우려에도 불구하고 북한이 6차 핵실험을 감행했습니다. 한반도의 긴...,"[국제사회의 제재와 우려에도 불구하고 북한이 6차 핵실험을 감행했습니다, 한반도의 ...",NaN,NaN,NaN
2,"과학기술정보통신부는 오늘 오전 9시10분쯤 지구 자기장, 즉 지자기의 강력한 교란이...",[과학기술정보통신부는 오늘 오전 9시10분쯤 지구 자기장 즉 지자기의 강력한 교란이...,NaN,NaN,NaN
3,중국의 대학 연구팀이 지난 3일 북한의 6차 핵실험 위력을 지진 관측 자료를 통해서...,[중국의 대학 연구팀이 지난 3일 북한의 6차 핵실험 위력을 지진 관측 자료를 통해...,NaN,NaN,NaN
4,김정은 집권 이후 북한이 부쩍 강조하고 있는 분야가 있습니다. 바로 과학기술입니다....,"[김정은 집권 이후 북한이 부쩍 강조하고 있는 분야가 있습니다, 바로 과학기술입니다...",NaN,NaN,NaN


In [16]:
tqdm.pandas()

In [17]:
for idx in tqdm(range(len(df_contents)), desc='tagging', total=(len(df_contents))):
    df_contents['tags'][idx] = k_analyze(df_contents['lines'][idx])
    df_contents['morphs'][idx], df_contents['mor_sep'][idx] = morph_analyze(df_contents['lines'][idx])
        

tagging:   0%|          | 0/29195 [00:00<?, ?it/s]/root/miniconda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/root/miniconda/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/root/miniconda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so 

In [ ]:
#df_contents['tags'] = df_contents['lines'].progress_apply(lambda x: k_analyze(x))

In [18]:
df_contents.head(5)

,texts,lines,tags,morphs,mor_sep
0,중국의 민간 기업이 미사일 개발에 필요한 소재를 북한에 수출했다고 아사히신문이 현지...,[중국의 민간 기업이 미사일 개발에 필요한 소재를 북한에 수출했다고 아사히신문이 현...,"[중국, 민간, 기업, 미사일, 개발, 필요, 소재, 북한, 수출, 아사히신, 문,...","[중국의\t중국/NNP + 의/JKG, 민간\t민간/NNG, 기업이\t기업/NNG ...","[중국/NNP, 의/JKG, 민간/NNG, 기업/NNG, 이/JKS, 미사일/NNG..."
1,국제사회의 제재와 우려에도 불구하고 북한이 6차 핵실험을 감행했습니다. 한반도의 긴...,"[국제사회의 제재와 우려에도 불구하고 북한이 6차 핵실험을 감행했습니다, 한반도의 ...","[국제, 사회, 제재, 우려, 북한, 핵실험, 감행, 한반도, 긴장감, 때, 오늘,...","[국제사회의\t국제/NNG + 사회/NNG + 의/JKG, 제재와\t제재/NNG +...","[국제/NNG, 사회/NNG, 의/JKG, 제재/NNG, 와/JC, 우려/NNG, ..."
2,"과학기술정보통신부는 오늘 오전 9시10분쯤 지구 자기장, 즉 지자기의 강력한 교란이...",[과학기술정보통신부는 오늘 오전 9시10분쯤 지구 자기장 즉 지자기의 강력한 교란이...,"[과학, 기술, 정보, 통신부, 오늘, 오전, 지구, 자기장, 지자기, 교란, 발생...",[과학기술정보통신부는\t과학/NNG + 기술/NNG + 정보/NNG + 통신부/NN...,"[과학/NNG, 기술/NNG, 정보/NNG, 통신부/NNG, 는/JX, 오늘/NNG..."
3,중국의 대학 연구팀이 지난 3일 북한의 6차 핵실험 위력을 지진 관측 자료를 통해서...,[중국의 대학 연구팀이 지난 3일 북한의 6차 핵실험 위력을 지진 관측 자료를 통해...,"[중국, 대학, 연구, 팀, 북한, 핵실험, 위력, 지진, 관측, 자료, 분석, 결...","[중국의\t중국/NNP + 의/JKG, 대학\t대학/NNG, 연구팀이\t연구/NNG...","[중국/NNP, 의/JKG, 대학/NNG, 연구/NNG, 팀/NNG, 이/JKS, ..."
4,김정은 집권 이후 북한이 부쩍 강조하고 있는 분야가 있습니다. 바로 과학기술입니다....,"[김정은 집권 이후 북한이 부쩍 강조하고 있는 분야가 있습니다, 바로 과학기술입니다...","[김정, 집권, 이후, 북한, 강조, 분야, 과학, 기술, 북한, 과학자, 우대, ...","[김정은\t김정/NNP + 은/JX, 집권\t집권/NNG, 이후\t이후/NNG, 북...","[김정/NNP, 은/JX, 집권/NNG, 이후/NNG, 북한/NNP, 이/JKS, ..."


In [22]:
df_contents.to_csv('data.csv', index=False)

In [23]:
df = pd.read_csv('data.csv')

In [24]:
df.head(5)

,texts,lines,tags,morphs,mor_sep
0,중국의 민간 기업이 미사일 개발에 필요한 소재를 북한에 수출했다고 아사히신문이 현지...,['중국의 민간 기업이 미사일 개발에 필요한 소재를 북한에 수출했다고 아사히신문이 ...,"['중국', '민간', '기업', '미사일', '개발', '필요', '소재', '북...","['중국의\t중국/NNP + 의/JKG', '민간\t민간/NNG', '기업이\t기업...","['중국/NNP', '의/JKG', '민간/NNG', '기업/NNG', '이/JKS..."
1,국제사회의 제재와 우려에도 불구하고 북한이 6차 핵실험을 감행했습니다. 한반도의 긴...,"['국제사회의 제재와 우려에도 불구하고 북한이 6차 핵실험을 감행했습니다', '한반...","['국제', '사회', '제재', '우려', '북한', '핵실험', '감행', '한...","['국제사회의\t국제/NNG + 사회/NNG + 의/JKG', '제재와\t제재/NN...","['국제/NNG', '사회/NNG', '의/JKG', '제재/NNG', '와/JC'..."
2,"과학기술정보통신부는 오늘 오전 9시10분쯤 지구 자기장, 즉 지자기의 강력한 교란이...",['과학기술정보통신부는 오늘 오전 9시10분쯤 지구 자기장 즉 지자기의 강력한 교란...,"['과학', '기술', '정보', '통신부', '오늘', '오전', '지구', '자...",['과학기술정보통신부는\t과학/NNG + 기술/NNG + 정보/NNG + 통신부/N...,"['과학/NNG', '기술/NNG', '정보/NNG', '통신부/NNG', '는/J..."
3,중국의 대학 연구팀이 지난 3일 북한의 6차 핵실험 위력을 지진 관측 자료를 통해서...,['중국의 대학 연구팀이 지난 3일 북한의 6차 핵실험 위력을 지진 관측 자료를 통...,"['중국', '대학', '연구', '팀', '북한', '핵실험', '위력', '지진...","['중국의\t중국/NNP + 의/JKG', '대학\t대학/NNG', '연구팀이\t연...","['중국/NNP', '의/JKG', '대학/NNG', '연구/NNG', '팀/NNG..."
4,김정은 집권 이후 북한이 부쩍 강조하고 있는 분야가 있습니다. 바로 과학기술입니다....,"['김정은 집권 이후 북한이 부쩍 강조하고 있는 분야가 있습니다', '바로 과학기술...","['김정', '집권', '이후', '북한', '강조', '분야', '과학', '기술...","['김정은\t김정/NNP + 은/JX', '집권\t집권/NNG', '이후\t이후/N...","['김정/NNP', '은/JX', '집권/NNG', '이후/NNG', '북한/NNP..."


In [89]:
df_contents['tags_set'] = df_contents['tags'].apply(lambda x: set(x))

### Counting

In [91]:
df_contents['counter'] = df_contents['tags'].apply(lambda x: Counter(x))
df_contents['counter2'] = df_contents['tags_set'].apply(lambda x: Counter(x))

In [34]:
df_contents.head()

,texts,lines,tags,morphs,mor_sep,counter
0,중국의 민간 기업이 미사일 개발에 필요한 소재를 북한에 수출했다고 아사히신문이 현지...,[중국의 민간 기업이 미사일 개발에 필요한 소재를 북한에 수출했다고 아사히신문이 현...,"[중국, 민간, 기업, 미사일, 개발, 필요, 소재, 북한, 수출, 아사히신, 문,...","[중국의\t중국/NNP + 의/JKG, 민간\t민간/NNG, 기업이\t기업/NNG ...","[중국/NNP, 의/JKG, 민간/NNG, 기업/NNG, 이/JKS, 미사일/NNG...","{'중국': 5, '민간': 2, '기업': 5, '미사일': 3, '개발': 3,..."
1,국제사회의 제재와 우려에도 불구하고 북한이 6차 핵실험을 감행했습니다. 한반도의 긴...,"[국제사회의 제재와 우려에도 불구하고 북한이 6차 핵실험을 감행했습니다, 한반도의 ...","[국제, 사회, 제재, 우려, 북한, 핵실험, 감행, 한반도, 긴장감, 때, 오늘,...","[국제사회의\t국제/NNG + 사회/NNG + 의/JKG, 제재와\t제재/NNG +...","[국제/NNG, 사회/NNG, 의/JKG, 제재/NNG, 와/JC, 우려/NNG, ...","{'국제': 5, '사회': 4, '제재': 13, '우려': 3, '북한': 64..."
2,"과학기술정보통신부는 오늘 오전 9시10분쯤 지구 자기장, 즉 지자기의 강력한 교란이...",[과학기술정보통신부는 오늘 오전 9시10분쯤 지구 자기장 즉 지자기의 강력한 교란이...,"[과학, 기술, 정보, 통신부, 오늘, 오전, 지구, 자기장, 지자기, 교란, 발생...",[과학기술정보통신부는\t과학/NNG + 기술/NNG + 정보/NNG + 통신부/NN...,"[과학/NNG, 기술/NNG, 정보/NNG, 통신부/NNG, 는/JX, 오늘/NNG...","{'과학': 1, '기술': 1, '정보': 1, '통신부': 1, '오늘': 1,..."
3,중국의 대학 연구팀이 지난 3일 북한의 6차 핵실험 위력을 지진 관측 자료를 통해서...,[중국의 대학 연구팀이 지난 3일 북한의 6차 핵실험 위력을 지진 관측 자료를 통해...,"[중국, 대학, 연구, 팀, 북한, 핵실험, 위력, 지진, 관측, 자료, 분석, 결...","[중국의\t중국/NNP + 의/JKG, 대학\t대학/NNG, 연구팀이\t연구/NNG...","[중국/NNP, 의/JKG, 대학/NNG, 연구/NNG, 팀/NNG, 이/JKS, ...","{'중국': 4, '대학': 2, '연구': 3, '팀': 3, '북한': 1, '..."
4,김정은 집권 이후 북한이 부쩍 강조하고 있는 분야가 있습니다. 바로 과학기술입니다....,"[김정은 집권 이후 북한이 부쩍 강조하고 있는 분야가 있습니다, 바로 과학기술입니다...","[김정, 집권, 이후, 북한, 강조, 분야, 과학, 기술, 북한, 과학자, 우대, ...","[김정은\t김정/NNP + 은/JX, 집권\t집권/NNG, 이후\t이후/NNG, 북...","[김정/NNP, 은/JX, 집권/NNG, 이후/NNG, 북한/NNP, 이/JKS, ...","{'김정': 5, '집권': 4, '이후': 3, '북한': 17, '강조': 5,..."


In [65]:
a = df_contents['counter'][0]
b = df_contents['counter'][1]

In [71]:
counter_all = Counter()
for idx in tqdm(range(len(df_contents)), desc="Outer Process", total=(len(df_contents))):
    counter_all += df_contents['counter'][idx]

Outer Process: 100%|██████████| 29195/29195 [04:09<00:00, 116.98it/s]


In [72]:
len(counter_all.keys())

121484

In [78]:
counter_all['중국']

4999

In [99]:
dict_counter = {}
for word in dict_ls:
    if word in counter_all.keys():
        dict_counter[word] = counter_all[word]

In [100]:
dict_counter['중국']

4999

In [101]:
df_result = pd.DataFrame.from_dict(dict_counter, orient='index', columns=['frequency'])

In [102]:
df_result.head()

,frequency
시간,8438
중국,4999
우리,123
발사,10076
정도,11756


In [92]:
counter_set = Counter()
for idx in tqdm(range(len(df_contents)), desc="Outer Process", total=(len(df_contents))):
    counter_set += df_contents['counter2'][idx]

Outer Process: 100%|██████████| 29195/29195 [02:51<00:00, 170.71it/s]


In [94]:
counter_set['플라나리아']

4

In [97]:
dict_set_counter = {}
for word in dict_ls:
    if word in counter_set.keys():
        dict_set_counter[word] = counter_set[word]

In [103]:
df_result2 = pd.DataFrame.from_dict(dict_set_counter, orient='index', columns=['frequency'])

In [104]:
df_result2.head()

,frequency
시간,4981
중국,1990
우리,98
발사,1795
정도,7341


In [88]:
df_result.to_excel('result.xlsx', sheet_name='sheet1')

In [105]:
df_result2.to_excel('result2.xlsx', sheet_name='sheet1')